In [1]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

import time
import json

In [2]:
def get_links(page):
    options = Options()
    options.add_argument('--ignore-certificate-errors')

    service = Service(executable_path=rf'C:\Users\syune\OneDrive\Desktop\chromedriver.exe')
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(page)
    driver.maximize_window()
    driver.execute_script("document.body.style.zoom='80%'")
    time.sleep(10)

    # Random click to close ad
    actions = ActionChains(driver)
    actions.move_by_offset(348 , 23).click().perform()
    actions.move_by_offset(-348, -23).perform()

    time.sleep(5)
    try:
        WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, '//button[@class="my_store_close_button"]'))
        )
        driver.find_element(By.XPATH, '//button[@class="my_store_close_button"]').click()

    except (NoSuchElementException, TimeoutException):
        pass 

    try:
        WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, '//button[@id="btn-cookie-allow"]'))
        )
        driver.find_element(By.XPATH, '//button[@id="btn-cookie-allow"]').click()

    except (NoSuchElementException, TimeoutException):
        pass 


    # Get to the end of page
    while True:
        try:
            try:
                iframe = driver.find_element(By.XPATH, '//button[@id="loadMore"]')
                # If hidden, break
                if "js-is-hidden" in iframe.get_attribute("class"):
                    print("Pagination is hidden. Stopping.")
                    break
                ActionChains(driver).scroll_to_element(iframe).perform()
            except NoSuchElementException:
                print("Pagination element not found while trying to scroll. Stopping.")
                break

            try:
                clickable = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[@id="loadMore"]'))
                )
                clickable.click()
                print("Clicked pagination.")
            except (TimeoutException, ElementClickInterceptedException):
                print("Pagination element not clickable or blocked. Stopping.")
                break

            time.sleep(1)  # small pause to ensure content loaded

        except Exception as e:
            print(f"Page finished.")
            break
    

    # Find all elements with class 'listGridItemInfo'
    elements = driver.find_elements(By.CLASS_NAME, "CoveoResultLink")

    # Extract href attribute from each element (assuming these elements have <a> tags or are <a> themselves)
    links = []

    for el in elements:
        # If the element itself is an <a> tag
        if el.tag_name == "a":
            href = el.get_attribute("href")
            if href:
                links.append(href)
        else:
            # If the <a> tag is inside this element
            a_tag = el.find_element(By.TAG_NAME, "a")
            href = a_tag.get_attribute("href")
            if href:
                links.append(href)

    links = links[0::3]

    return driver, links

In [3]:
def get_info_and_picture(driver, link, folder):
    driver.get(link)
    time.sleep(3)
    
    driver.execute_script("window.scrollBy(0, 480);")

    product_id = link.rsplit('-', 1)[-1]

    name = driver.find_element(By.XPATH, '//div[@class="page-title-wrapper product"]').text

    # Default values as None
    region = None
    brand = None
    type = None

    # Get all label elements
    labels = driver.find_elements(By.CLASS_NAME, "label")

    for label in labels:
        label_text = label.text.strip()

        try:
            value = label.find_element(By.XPATH, "./following-sibling::div[@class='value']").text.strip()
        except:
            value = None

        if label_text == "Made In":
            region = value
        elif label_text == "By":
            brand = value
        elif label_text == "Varietal":
            type = value

    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@class="fotorama__img"]'))
    )

    with open(f'C:/Users/syune/OneDrive/Desktop/Wine_Data/Data/{folder}/images/{product_id}.png', 'wb') as file:
        file.write(element.screenshot_as_png)

    wine_data = {
        "Name": name,
        "Brand": brand,
        "Type": type,
        "Region": region
    }

    with open(f'C:/Users/syune/OneDrive/Desktop/Wine_Data/Data/{folder}/labels/{product_id}.json', "w", encoding="utf-8") as f:
        json.dump(wine_data, f, indent=4, ensure_ascii=False)

### Rose

In [13]:
all_pages = ["https://www.lcbo.com/en/products/wine/rose-wine#t=clp-products-wine-ros%C3%A9_wine&sort=relevancy&layout=card"]

In [14]:
for x in range(0,len(all_pages)):
    print(all_pages[x])
    page = all_pages[x]
    driver, links = get_links(page)
    print(x, " pages link count - ", len(links))
    for i in range(0, len(links)):
        link = links[i]
        get_info_and_picture(driver, link, folder='Rose')
    
    print('Page done', x, ". Last item was: ", i)
    driver.close()

https://www.lcbo.com/en/products/wine/rose-wine#t=clp-products-wine-ros%C3%A9_wine&sort=relevancy&layout=card
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Page finished.
0  pages link count -  354
Page done 0 . Last item was:  353


### Sparkling

In [15]:
all_pages = ["https://www.lcbo.com/en/products/wine/sparkling-wine#t=clp-products-wine-sparkling_wine&sort=relevancy&layout=card"]

In [16]:
for x in range(0,len(all_pages)):
    print(all_pages[x])
    page = all_pages[x]
    driver, links = get_links(page)
    print(x, " pages link count - ", len(links))
    for i in range(0, len(links)):
        link = links[i]
        get_info_and_picture(driver, link, folder='Sparkling')
    
    print('Page done', x, ". Last item was: ", i)
driver.close()

https://www.lcbo.com/en/products/wine/sparkling-wine#t=clp-products-wine-sparkling_wine&sort=relevancy&layout=card
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Page finished.
0  pages link count -  464
Page done 0 . Last item was:  463


### Champagne

In [17]:
all_pages = ["https://www.lcbo.com/en/products/wine/champagne#t=clp-products-wine-champagne&sort=relevancy&layout=card"]

In [19]:
for x in range(0,len(all_pages)):
    print(all_pages[x])
    page = all_pages[x]
    driver, links = get_links(page)
    print(x, " pages link count - ", len(links))
    for i in range(0, len(links)):
        link = links[i]
        get_info_and_picture(driver, link, folder='Champagne')
    
    print('Page done', x, ". Last item was: ", i)
    driver.close()

https://www.lcbo.com/en/products/wine/champagne#t=clp-products-wine-champagne&sort=relevancy&layout=card
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Page finished.
0  pages link count -  210
Page done 0 . Last item was:  209


### Fortified

In [20]:
all_pages = ["https://www.lcbo.com/en/products/wine/fortified-wine#t=clp-products-wine-fortified_wine&sort=relevancy&layout=card"]

In [21]:
for x in range(0,len(all_pages)):
    print(all_pages[x])
    page = all_pages[x]
    driver, links = get_links(page)
    print(x, " pages link count - ", len(links))
    for i in range(0, len(links)):
        link = links[i]
        get_info_and_picture(driver, link, folder='Fortified')
    
    print('Page done', x, ". Last item was: ", i)
    driver.close()

https://www.lcbo.com/en/products/wine/fortified-wine#t=clp-products-wine-fortified_wine&sort=relevancy&layout=card
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Page finished.
0  pages link count -  199
Page done 0 . Last item was:  198


### Sake_Rice_wine

In [22]:
all_pages = ["https://www.lcbo.com/en/products/beer-cider/specialty-beer/sake-rice-wine#t=clp-products-wine-sake_%26_rice_wine&sort=relevancy&layout=card"]

In [23]:
for x in range(0,len(all_pages)):
    print(all_pages[x])
    page = all_pages[x]
    driver, links = get_links(page)
    print(x, " pages link count - ", len(links))
    for i in range(0, len(links)):
        link = links[i]
        get_info_and_picture(driver, link, folder='Sake_Rice_wine')
    
    print('Page done', x, ". Last item was: ", i)
    driver.close()

https://www.lcbo.com/en/products/beer-cider/specialty-beer/sake-rice-wine#t=clp-products-wine-sake_%26_rice_wine&sort=relevancy&layout=card
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Page finished.
0  pages link count -  356
Page done 0 . Last item was:  355


### Icewine

In [24]:
all_pages = ["https://www.lcbo.com/en/products/wine/icewine#t=clp-products-wine-icewine&sort=relevancy&layout=card"]

In [25]:
for x in range(0,len(all_pages)):
    print(all_pages[x])
    page = all_pages[x]
    driver, links = get_links(page)
    print(x, " pages link count - ", len(links))
    for i in range(0, len(links)):
        link = links[i]
        get_info_and_picture(driver, link, folder='Icewine')
    
    print('Page done', x, ". Last item was: ", i)
    driver.close()

https://www.lcbo.com/en/products/wine/icewine#t=clp-products-wine-icewine&sort=relevancy&layout=card
Clicked pagination.
Page finished.
0  pages link count -  51
Page done 0 . Last item was:  50


### Specialty

In [26]:
all_pages = ["https://www.lcbo.com/en/products/wine/specialty-wines#t=clp-products-wine-specialty_wines&sort=relevancy&layout=card"]

In [27]:
for x in range(0,len(all_pages)):
    print(all_pages[x])
    page = all_pages[x]
    driver, links = get_links(page)
    print(x, " pages link count - ", len(links))
    for i in range(0, len(links)):
        link = links[i]
        get_info_and_picture(driver, link, folder='Specialty')
    
    print('Page done', x, ". Last item was: ", i)
    driver.close()

https://www.lcbo.com/en/products/wine/specialty-wines#t=clp-products-wine-specialty_wines&sort=relevancy&layout=card
Clicked pagination.
Clicked pagination.
Page finished.
0  pages link count -  144
Page done 0 . Last item was:  143


### Dessert

In [28]:
all_pages = ["https://www.lcbo.com/en/products/wine/dessert-wine#t=clp-products-wine-dessert_wine&sort=relevancy&layout=card"]

In [29]:
for x in range(0,len(all_pages)):
    print(all_pages[x])
    page = all_pages[x]
    driver, links = get_links(page)
    print(x, " pages link count - ", len(links))
    for i in range(0, len(links)):
        link = links[i]
        get_info_and_picture(driver, link, folder='Dessert')
    
    print('Page done', x, ". Last item was: ", i)
    driver.close()

https://www.lcbo.com/en/products/wine/dessert-wine#t=clp-products-wine-dessert_wine&sort=relevancy&layout=card
Page finished.
0  pages link count -  29
Page done 0 . Last item was:  28


### White

In [4]:
all_pages = ["https://www.lcbo.com/en/products/wine/white-wine#t=clp-products-wine-white_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[France]", 
             'https://www.lcbo.com/en/products/wine/white-wine#t=clp-products-wine-white_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[Canada]',
             "https://www.lcbo.com/en/products/wine/white-wine#t=clp-products-wine-white_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[Italy,Greece]",
             'https://www.lcbo.com/en/products/wine/white-wine#t=clp-products-wine-white_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[New%20Zealand,United%20States,Portugal]']

In [5]:
for x in range(0,len(all_pages)):
    print(all_pages[x])
    page = all_pages[x]
    driver, links = get_links(page)
    print(x, " pages link count - ", len(links))
    if len(links) >= 200:
        lenght = len(links) / 3
        for i in range(0, int(lenght)):
            link = links[i]
            get_info_and_picture(driver, link, folder='White')
    else:
        for i in range(0, len(links)):
            link = links[i]
            get_info_and_picture(driver, link, folder='White')
    
    print('Page done', x, ". Last item was: ", i)
    driver.close()

https://www.lcbo.com/en/products/wine/white-wine#t=clp-products-wine-white_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[France]
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Page finished.
0  pages link count -  548
Page done 0 . Last item was:  181
https://www.lcbo.com/en/products/wine/white-wine#t=clp-products-wine-white_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[Canada]
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Page finished.
1  pages link count -  465
Page done 1 . Last item was:  154
https://www.lcbo.com/en/products/wine/white-wine#t=clp-products-wine-white_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[Italy,Greece]
Clicked paginati

In [6]:
all_pages = ['https://www.lcbo.com/en/products/wine/white-wine#t=clp-products-wine-white_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[Germany,South%20Africa,Australia,Spain,Austria]',
             "https://www.lcbo.com/en/products/wine/white-wine#t=clp-products-wine-white_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[Chile,Argentina,Hungary,Georgia,Israel,Croatia,Slovenia]",
             'https://www.lcbo.com/en/products/wine/white-wine#t=clp-products-wine-white_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[Armenia,Romania,Lebanon,Uruguay,Ukraine,China,Moldova,India,Serbia]']

In [7]:
for x in range(0,len(all_pages)):
    print(all_pages[x])
    page = all_pages[x]
    driver, links = get_links(page)
    print(x, " pages link count - ", len(links))
    if len(links) >= 100:
        lenght = len(links) / 2
        for i in range(0, int(lenght)):
            link = links[i]
            get_info_and_picture(driver, link, folder='White')
    
    else:
        for i in range(0, len(links)):
            link = links[i]
            get_info_and_picture(driver, link, folder='White')
    
    print('Page done', x, ". Last item was: ", i)
    driver.close()

https://www.lcbo.com/en/products/wine/white-wine#t=clp-products-wine-white_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[Germany,South%20Africa,Australia,Spain,Austria]
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Page finished.
0  pages link count -  487
Page done 0 . Last item was:  242
https://www.lcbo.com/en/products/wine/white-wine#t=clp-products-wine-white_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[Chile,Argentina,Hungary,Georgia,Israel,Croatia,Slovenia]
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Page finished.
1  pages link count -  256
Page done 1 . Last item was:  127
https://www.lcbo.com/en/products/wine/white-wine#t=clp-products-wine-white_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[Armenia,Romania,Lebanon,Uruguay,Ukraine,C

### Red

In [8]:
all_pages = ["https://www.lcbo.com/en/products/wine/red-wine#t=clp-products-wine-red_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[France]&f:@lcbo_region_name=[Bordeaux]",
             "https://www.lcbo.com/en/products/wine/red-wine#t=clp-products-wine-red_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[France]&f:@lcbo_region_name=[Burgundy,Rh%C3%B4ne]",
             "https://www.lcbo.com/en/products/wine/red-wine#t=clp-products-wine-red_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[France]&f:@lcbo_region_name=[Languedoc,Beaujolais,South%20of%20France,Loire]",
             "https://www.lcbo.com/en/products/wine/red-wine#t=clp-products-wine-red_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[Italy]&f:@lcbo_region_name=[Tuscany]",
             "https://www.lcbo.com/en/products/wine/red-wine#t=clp-products-wine-red_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[Italy]&f:@lcbo_region_name=[Piedmont,Veneto,Sicily,Puglia,Abruzzo,Umbria]",
             "https://www.lcbo.com/en/products/wine/red-wine#t=clp-products-wine-red_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[Canada]",
             "https://www.lcbo.com/en/products/wine/red-wine#t=clp-products-wine-red_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[Australia]",
             "https://www.lcbo.com/en/products/wine/red-wine#t=clp-products-wine-red_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[United%20States]",
             "https://www.lcbo.com/en/products/wine/red-wine#t=clp-products-wine-red_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[Spain]",
             "https://www.lcbo.com/en/products/wine/red-wine#t=clp-products-wine-red_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[Portugal]",
             "https://www.lcbo.com/en/products/wine/red-wine#t=clp-products-wine-red_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[Argentina]",
             "https://www.lcbo.com/en/products/wine/red-wine#t=clp-products-wine-red_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[Chile,Greece]",
             "https://www.lcbo.com/en/products/wine/red-wine#t=clp-products-wine-red_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[South%20Africa,Israel,New%20Zealand,Georgia]",
             "https://www.lcbo.com/en/products/wine/red-wine#t=clp-products-wine-red_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[Croatia,Austria,Hungary,Germany,Armenia,Lebanon]"]

In [9]:
for x in range(0,len(all_pages)):
    print(all_pages[x])
    page = all_pages[x]
    driver, links = get_links(page)
    print(x, " pages link count - ", len(links))
    if len(links) >= 100:
        lenght = len(links) / 3
        for i in range(0, int(lenght)):
            link = links[i]
            get_info_and_picture(driver, link, folder='Red')
    
    else:
        for i in range(0, len(links)):
            link = links[i]
            get_info_and_picture(driver, link, folder='Red')
    
    print('Page done', x, ". Last item was: ", i)
    driver.close()

https://www.lcbo.com/en/products/wine/red-wine#t=clp-products-wine-red_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[France]&f:@lcbo_region_name=[Bordeaux]
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Page finished.
0  pages link count -  500
Page done 0 . Last item was:  165
https://www.lcbo.com/en/products/wine/red-wine#t=clp-products-wine-red_wine&sort=relevancy&layout=card&f:@country_of_manufacture=[France]&f:@lcbo_region_name=[Burgundy,Rh%C3%B4ne]
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Clicked pagination.
Page finished.
1  pages link count -  473
Page done 1 . Last item was:  156
https://www.lcbo.com/en/products/wine/red-wine#t=clp-products-wine-red_wine&sort=relevancy&layout=card&f:@country_of_ma